# Financial Document Analysis with Jamba 1.5 Large/Mini on Bedrock 
Jamba 1.5 models can be used to analyze financial documents. In this notebook, we will first load the Amazon's 10K filing (downloaded from [here](https://d18rn0p25nwr6d.cloudfront.net) and can then ask questions about it. This report has about 40,000 words.

### Install required libraries

In [ ]:
!pip install --upgrade boto3 --quiet

In [ ]:
#model ids

jamba_large="ai21.jamba-1-5-mini-v1:0"
jamba_mini="ai21.jamba-1-5-large-v1:0"
DEFAULT_MODEL= jamba_large

In [6]:
# Required imports
import os
import json
import requests
import boto3
import time

# Function to read the contents of the files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Paths to the text files
document = '10k.txt'

# Function to read the contents of the files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def converse_jamba(prompt,**kwargs):
    bedrock = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")
    inference_config = {"temperature": 0.7, "maxTokens": 4000, "topP": 0.8}
    inference_config.update(kwargs)
    
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    modelId = DEFAULT_MODEL
    response = bedrock.converse(
        messages=messages,
        modelId=modelId,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    return generated_text, response

def call_bedrock_jamba(prompt, **kwargs):
    attempts = 0
    while attempts < 5:
        try:
            return converse_jamba(prompt, **kwargs)
        except Exception as e:
            print(e)
            attempts += 1
            print(f"Attempt {attempts}: Failed to call API, retrying in 3 seconds...")
            time.sleep(3)
    raise Exception("Failed to complete the API call after 5 attempts")

### Ask a Question
Next we will ask a question from the document, passing the entire content of the document to the prompt. 

In [7]:

question="What was Amazon's revenue generating activity in 2019?"
document_content=read_file(document)
q_a_prompt = f"""
You are an excellent research assistant. Based the following "Document Content" do your best to answer the question posed.
Keep your answer strictly grounded in the document, and if the answer cannot be found in the document, just say "I do not know"

Document Content:

{document_content}

Question: 
{question}

Answer to Question:
"""

#print(comparison_prompt)
# Call the API

#print(q_a_prompt)


answer = call_bedrock_jamba(q_a_prompt,temperature=0.7)

# Print the comparison result
print("Answer:")
print(answer)


Answer:
(" In 2019, Amazon's revenue generating activities included retail sales of consumer products, subscriptions, and third-party seller services. These activities were primarily conducted through both online and physical stores.", {'ResponseMetadata': {'RequestId': 'c27923fa-c7c1-4577-89b5-e1691606b333', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 21 Sep 2024 18:15:09 GMT', 'content-type': 'application/json', 'content-length': '411', 'connection': 'keep-alive', 'x-amzn-requestid': 'c27923fa-c7c1-4577-89b5-e1691606b333'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': " In 2019, Amazon's revenue generating activities included retail sales of consumer products, subscriptions, and third-party seller services. These activities were primarily conducted through both online and physical stores."}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 65374, 'outputTokens': 45, 'totalTokens': 65419}, 'metrics': {'latencyMs': 2741}})
